In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('S30 ETL Assignment.db')
cursor = conn.cursor()

# Get table names from DB
table_names= """
SELECT name FROM sqlite_master WHERE type='table';
"""
cursor.execute(table_names)
tables_list = cursor.fetchall()
print(tables_list)

# Result Query

sql = """
select c.customer_id,c.age,o.item_id,sum(o.quantity) as sum from customers c 
join sales s on c.customer_id = s.customer_id
join orders o on s.sales_id = o.sales_id
join items i on o.item_id = i.item_id
where c.age >=18 and c.age <= 35 and quantity is not null
group by 1,2,3 
"""

cursor.execute(sql)
result = cursor.fetchall()

df = pd.DataFrame(result,columns = ['customer_id','age','item_id','Total quantity'])
df['Total quantity'] = df['Total quantity'].astype(int)
df.to_csv('output_file.csv',sep = ';',index = False)

# Close the database connection
conn.close()


[('sqlite_sequence',), ('sales',), ('orders',), ('items',), ('customers',)]


In [3]:
# Pandas Solution

# Connect to the SQLite database
conn = sqlite3.connect('S30 ETL Assignment.db')
cursor = conn.cursor()

# Customers DataFrame
customers_sql = """
select * from customers
"""
cursor.execute(customers_sql)
customers_df = pd.DataFrame(cursor.fetchall(),columns = ['customer_id','age'])

# Sales DataFrame
sales_sql = """
select * from sales;
"""
cursor.execute(sales_sql)
sales_df = pd.DataFrame(cursor.fetchall(),columns = ['sales_id','customer_id'])

# Orders DataFrame
orders_sql = """
select * from orders;
"""
cursor.execute(orders_sql)
orders_df = pd.DataFrame(cursor.fetchall(),columns = ['order_id','sales_id','item_id','quantity'])

# Items DataFrame
items_sql = """
select * from items;
"""
cursor.execute(items_sql)
items_df  = pd.DataFrame(cursor.fetchall(),columns = ['item_id','item_name'])

joined_df = customers_df.merge(sales_df,on='customer_id').merge(orders_df,on='sales_id').merge(items_df,on='item_id')
joined_df = joined_df[['customer_id', 'age', 'item_id','quantity']]
filtered_df = joined_df[(joined_df['quantity'].notnull()) & (joined_df['age'].between(18, 35))]
result_df = filtered_df.groupby(['customer_id', 'age', 'item_id'])['quantity'].sum().reset_index()

# Rename the sum column to 'total_quantity' (optional)
result_df = result_df.rename(columns={'quantity': 'Total quantity'})
result_df['Total quantity'] = df['Total quantity'].astype(int)
result_df.to_csv('output_file_pandas.csv',sep = ';',index = False)